In [1]:
import geopandas
from openai import OpenAI

import os
import wget
import zipfile

from dotenv import load_dotenv

from hackathonopenai.paleontogical_potential import PaleontogicalPotentialAssistant

%load_ext autoreload
%autoreload 2

In [2]:
def download_paleontological_potential_data(
        url_path: str = "https://opendata.arcgis.com/datasets/bd874a0d01674a079a611263a463f97f_0.zip",
        output_directory_base: str = "../data/potencial_paleontologico",
        raw_data: str = '../data/raw',
        name: str = "data_pot_paleon"
) -> str:
    """
    Download the paleontological potential data from the url_path and save it in the output_directory_base
    """
    if not os.path.exists(output_directory_base):
        os.makedirs(output_directory_base, exist_ok=True)

    if not os.path.exists(raw_data):
        os.makedirs(raw_data, exist_ok=True)

    # Download the data
    output_directory = wget.download(url_path, out=raw_data)
    # Unzip the data
    with zipfile.ZipFile(output_directory, 'r') as zip_ref:
        zip_ref.extractall(output_directory_base)

    for file in os.listdir(output_directory_base):
        os.rename(
            os.path.join(output_directory_base, file), 
            os.path.join(output_directory_base, name + os.path.splitext(file)[1])
        )
    return output_directory_base

In [3]:
load_dotenv("../.env")
name = "data_pot_paleon"
output_directory_base = download_paleontological_potential_data(name=name)

# Load files
df = geopandas.read_file(
    os.path.join(output_directory_base, name + ".shp")
)
df_demo = geopandas.read_file(
    "../data/demo_data/doc.kml"
)

-1 / unknown

In [4]:
from shapely.ops import unary_union

single_polygon = unary_union(df_demo.geometry)
single_polygon_df = geopandas.GeoDataFrame(geometry=[single_polygon], crs=df_demo.crs)

In [6]:
df_kms = df.to_crs("EPSG:32633")
demo_geometry_kms = single_polygon_df.to_crs("EPSG:32633")

In [9]:
import json
import geopandas 
import folium

from openai import OpenAI
from shapely.geometry import Polygon
import streamlit as st

In [21]:
class PaleontogicalPotentialAssistant:

    def __init__(self, df: geopandas.GeoDataFrame, client: OpenAI):
        self.df = df
        self.client = client

    def system_prompt(self) -> str:
        
        
        system_prompt = """
        Eres un experto en potencial paleontologico y te han contratado para hacer la evaluación de impacto ambiental
        de un nuevo proyecto fotovoltaico en Chile.

        El usuario entregará su locacion y los suelos con detalle de potencial paleontologico con los cuales se solapa.

        Tu trabajo es entregar una detallada evaluación de impacto ambiental del proyecto fotovoltaico en esta zona, tomando en cuenta el tipo de potencial.

        Para esto primero vas a entregar un resumen si encuentras algo critico, para rechazar el proyecto, si no di que todo esta bien.

        En caso de que encuentres algo critico, debes entregar una evaluación detallada de los impactos ambientales y sociales del proyecto.

        El output será un JSON con dos campos
        ```json
        {
            "resumen": Resumen corto de la evaluacion,
            "evaluacion": Detalle de la evaluacion
        }
        ```
        """
        return system_prompt

    def format_message(self, overlap_zones: geopandas.GeoDataFrame) -> str:
        overlap_zones_data = str(overlap_zones.to_dict("records"))
        message = """
        las zonas paleontologicas con las que se solapa el suelo son:
        {overlap_zones}
        """
        return message.replace("{overlap_zones}", overlap_zones_data)


    def evaluate_project(self, location: Polygon) -> dict:
            overlap = self.df.geometry.overlaps(location.iloc[0].geometry)
    
            overlap_zones = self.df[ overlap == True]
            
            print("Numero de zonas solapadas: ", overlap_zones.shape[0])
    
            system_prompt = self.system_prompt()
            message = self.format_message(overlap_zones)
    
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": message}
            ]
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                response_format={ "type": "json_object" },
                temperature=0
            )
            return json.loads(response.choices[0].message.content)

In [23]:
paleontogical_assistant = PaleontogicalPotentialAssistant(df_kms, OpenAI(api_key=key))

In [24]:
response = paleontogical_assistant.evaluate_project(demo_geometry_kms)

Numero de zonas solapadas:  4


In [25]:
response

{'resumen': 'El proyecto fotovoltaico se solapa con áreas de potencial paleontológico susceptible, lo que podría generar impactos negativos en el patrimonio paleontológico. Se recomienda una evaluación más detallada antes de proceder.',
 'evaluacion': {'impactos_ambientales': {'paleontológicos': {'descripcion': "Las áreas de 'Terrazas de abrasión' y 'Depósitos coluviales' tienen un potencial paleontológico susceptible, lo que indica que podrían contener restos fósiles o información geológica relevante. La construcción del proyecto podría dañar o destruir estos recursos.",
    'medidas_mitigacion': ['Realizar un estudio paleontológico previo a la construcción para identificar y documentar los recursos presentes.',
     'Implementar un plan de manejo para la conservación de los hallazgos paleontológicos durante la construcción.',
     'Establecer un protocolo de rescate de fósiles en caso de hallazgos durante las obras.']}},
  'impactos_sociales': {'descripcion': 'La posible afectación d